In [1]:
import os

import openai
os.environ['OPENAI_API_KEY'] = "sk-48characterstofakeanopenaikey48charactersopenai0"
os.environ['OPENAI_API_BASE'] = "http://10.0.0.222:30307/v1"
openai.api_key = "sk-48characterstofakeanopenaikey48charactersopenai0"
openai.api_base = "http://10.0.0.222:30307/v1"

model = "Writer/camel-5b-hf"
#model = "mosaicml/mpt-7b-instruct"
#model = "mosaicml/mpt-30b-instruct"

INDEX_NAME = f"{model.replace('/', '-')}-default-coref"
persist_path = f"storage/{INDEX_NAME}"

In [2]:
import logging
import sys

#kron extensions to llama_index to support openai compatible api
sys.path.append('../llama-index/')

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Using Knowledge Graph

In [3]:
### Query Knowledge Graph

In [4]:

from llama_index import StorageContext
from llama_index import SimpleDirectoryReader, ServiceContext
from llama_index.graph_stores import SimpleGraphStore 
from llama_index import load_index_from_storage 
from llama_index.langchain_helpers.text_splitter import SentenceSplitter
from llama_index.node_parser import SimpleNodeParser

import tiktoken

#extensions to llama_index to support openai compatible endpoints, e.g. llama-api
from kron.llm_predictor.KronOpenAILLM import KronOpenAI
from kron.llm_predictor.KronLLMPredictor import KronLLMPredictor
#from kron.indices.knowledge_graph.KronKnowledgeGraphIndex import KronKnowledgeGraphIndex 
#from kron.prompts.kg_prompts import KRON_KG_TRIPLET_EXTRACT_PROMPT

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [5]:
#writer/camel uses endoftext 
from llama_index.utils import globals_helper
enc = tiktoken.get_encoding("gpt2")
tokenizer = lambda text: enc.encode(text, allowed_special={"<|endoftext|>"})
globals_helper._tokenizer = tokenizer

In [6]:
# define LLM
llm=KronOpenAI(temperature=0.01, model=model)
llm.max_tokens = 1536 
llm_predictor = KronLLMPredictor(llm)
print(llm_predictor.metadata)

context_window=2048 num_output=1536 is_chat_model=False model_name='unknown'


In [7]:
# define TextSplitter
text_splitter = SentenceSplitter(chunk_size=192, chunk_overlap=48, paragraph_separator='\n')
#define NodeParser
node_parser = SimpleNodeParser(text_splitter=text_splitter)
#define ServiceContext
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, node_parser=node_parser)

In [8]:
print(f'Loading index from {persist_path}')
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=persist_path)
# load index
index = load_index_from_storage(storage_context=storage_context, 
                                    service_context=service_context, 
                                    max_triplets_per_chunk=2,
                                    show_progress = True)

Loading index from storage/Writer-camel-5b-hf-default-coref
INFO:llama_index.indices.loading:Loading all indices.


In [9]:
#writer/camel does not understand the refine prompt
RESPONSE_MODE = 'accumulate'
query_engine = index.as_query_engine(response_mode = RESPONSE_MODE)

In [10]:
response = query_engine.query("What is SPGC?")
print(response)

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: What is SPGC?
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['Sustainable', 'SPGC stands for Sustainable Palm Oil.', 'stands', 'Palm', 'Oil', 'SPGC']
ERROR:llama_index.indices.knowledge_graph.retriever:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: 4dead89b-e5c4-4284-b193-8b564e971b1b: Tom Brown, Benjamin Mann, Nick Ryder, Melanie Subbiah, Jared  Kaplan, Prafull...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: 7434ab6d-ccfd-4f6c-9ee9-8ad16090f13b: This SPGC, “ADReSS: Multilingual Alzheimer’ Dementia Recognition through Spon...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: 2dec4db8-8b9b-4418-a26d-90c96904802e: THE PREDICTION TASKS This SPGC, “ADReSS: Multilingual Alzheimer’ Dementia Rec...
INFO:llama_index.indices.knowledge_graph.retriever:> Extracted

In [11]:
response = query_engine.query("What is AD recognition?")
print(response)

INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: What is AD recognition?
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['identifying', 'recognition', 'AD', 'disease', 'refers', 'diagnosing', 'individuals', 'process', 'Disease', 'related', "AD (Alzheimer's Disease) recognition refers to the process of identifying and diagnosing individuals with Alzheimer's disease or related disorders.", 'disorders', 'Alzheimer']
ERROR:llama_index.indices.knowledge_graph.retriever:Index was not constructed with embeddings, skipping embedding usage...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: fc7b0d90-1f97-42e1-bcb7-33212d0bc454: In Proceedings of the  Annual Meeting of the Association for Computational Li...
INFO:llama_index.indices.knowledge_graph.retriever:> Querying with idx: 5485e224-93ee-4e40-bb75-2c1ae8592d59: Maya Varma, Laurel Orr, Sen Wu, Megan Leszczynski, Xiao Ling, and Christopher...
INFO:llama_index.indices.knowledge_g

In [12]:
response = query_engine.query("What is the relationship between AD recognition and SPGC?")
print(response)


INFO:llama_index.indices.knowledge_graph.retriever:> Starting query: What is the relationship between AD recognition and SPGC?
INFO:llama_index.indices.knowledge_graph.retriever:> Query keywords: ['treatments', 'symptoms', 'validate', 'research', 'SPGC', 'patients', 'based', 'technique', 'medical', 'project', 'history', 'involving', 'multiple', 'working', 'new', 'develop', 'Alzheimer', 'used', 'recognition', 'identify', 'organizations', 'AD', 'disease', 'tools', 'classify', 'diagnostic', 'together', 'collaborative', "AD recognition is a technique used to identify and classify Alzheimer's disease patients based on their symptoms and medical history. SPGC is a collaborative research project involving multiple institutions and organizations working together to develop and validate new treatments and diagnostic tools for Alzheimer's disease.", 'institutions']
ERROR:llama_index.indices.knowledge_graph.retriever:Index was not constructed with embeddings, skipping embedding usage...
INFO:llam